In [1]:
import torch

In [15]:
prof = torch.profiler.profile(
    schedule=torch.profiler.schedule(wait=0, warmup=0, active=1, repeat=1),
    on_trace_ready=torch.profiler.tensorboard_trace_handler('log/perf'),
    record_shapes=True,
    with_stack=True,
    profile_memory=True)

device = torch.device('cuda:0')

In [16]:
prof.start()

s1 = torch.cuda.Stream(device=device)
s2 = torch.cuda.Stream(device=device)

a = torch.rand(7500, 7500, device=device)

with torch.cuda.stream(s1):
    torch.matmul(a, a)

with torch.cuda.stream(s2):
    torch.matmul(a, a)

torch.cuda.synchronize(device=device)

prof.step()
prof.stop()

In [ ]:
num_jobs_per_env = torch.arange(16)

num_total_jobs = num_jobs_per_env.sum().item()

worker_actions = torch.arange(50)
worker_actions = worker_actions.repeat(num_total_jobs).unsqueeze(1)

worker_actions